In [7]:
import pandas as pd
import numpy as np
from time import sleep
import os.path

In [15]:
artigos = pd.read_csv("titulo_doc_comunidade.csv",delimiter=';',quotechar='\"')
artigos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571 entries, 0 to 570
Data columns (total 3 columns):
title        571 non-null object
community    571 non-null int64
id           571 non-null object
dtypes: int64(1), object(2)
memory usage: 13.5+ KB


In [16]:
artigos.title.duplicated().sum()

0

In [25]:
artigos['downloaded'] = artigos.id.apply(lambda x: os.path.isfile('pdf/{}.pdf'.format(x)))

In [26]:
artigos[~artigos.downloaded]

,title,community,id,downloaded
87,Seminormal forms and cyclotomic quiver Hecke a...,6,1304.0906v1,False
209,Proceedings of the Twenty-Seventh Conference o...,104,1205.2596v2,False
373,Proceedings of the 29th International Conferen...,151,1207.4676v2,False


In [27]:
for ID in artigos[~artigos.downloaded].id.unique():
    link = "https://arxiv.org/pdf/"+ID+".pdf"
    ! cd pdf && curl -s -O {link} 
    sleep(np.random.random_sample()*35)

In [24]:
for ID in artigos.id.unique():
    pdf = "pdf/{}.pdf".format(ID)
    txt = "txt/{}.txt".format(ID)
    !pdftotext {pdf} {txt}

Internal Error: xref num 164 not found but needed, try to reconstruct<0a>
Internal Error: xref num 382 not found but needed, try to reconstruct<0a>
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error (2): Illegal character <3f> in hex string
Syntax Error (3): Illegal character <78> in hex string
Syntax Error (4): Illegal character <6d> in hex string
Syntax Error (5): Illegal character <6c> in hex string
Syntax Error (7): Illegal character <76> in hex string
Syntax Error (9): Illegal character <72> in hex string
Syntax Error (10): Illegal character <73> in hex string
Syntax Error (11): Illegal character <69> in hex string
Syntax Error (12): Illegal character <6f> in hex string
Syntax Error (13): Illegal character <6e> in hex string
Syntax Error (14): Illegal character <3d> in hex string
Syntax Error (15): Illegal character <22> in hex string
Syntax Error (17): Illegal character <2e> in hex string
Syntax Error (19): Illegal character <22> in hex string
Syntax Error (22

Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error (2): Illegal character <3f> in hex string
Syntax Error (3): Illegal character <78> in hex string
Syntax Error (4): Illegal character <6d> in hex string
Syntax Error (5): Illegal character <6c> in hex string
Syntax Error (7): Illegal character <76> in hex string
Syntax Error (9): Illegal character <72> in hex string
Syntax Error (10): Illegal character <73> in hex string
Syntax Error (11): Illegal character <69> in hex string
Syntax Error (12): Illegal character <6f> in hex string
Syntax Error (13): Illegal character <6e> in hex string
Syntax Error (14): Illegal character <3d> in hex string
Syntax Error (15): Illegal character <22> in hex string
Syntax Error (17): Illegal character <2e> in hex string
Syntax Error (19): Illegal character <22> in hex string
Syntax Error (22): Illegal character <6e> in hex string
Syntax Error (24): Illegal character <6f> in hex string
Syntax Error (26): Illegal character <69> in hex str

Syntax Error: Expected the default config, but wasn't able to find it, or it isn't a Dictionary
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error (2): Illegal character <3f> in hex string
Syntax Error (3): Illegal character <78> in hex string
Syntax Error (4): Illegal character <6d> in hex string
Syntax Error (5): Illegal character <6c> in hex string
Syntax Error (7): Illegal character <76> in hex string
Syntax Error (9): Illegal character <72> in hex string
Syntax Error (10): Illegal character <73> in hex string
Syntax Error (11): Illegal character <69> in hex string
Syntax Error (12): Illegal character <6f> in hex string
Syntax Error (13): Illegal character <6e> in hex string
Syntax Error (14): Illegal character <3d> in hex string
Syntax Error (15): Illegal character <22> in hex string
Syntax Error (17): Illegal character <2e> in hex string
Syntax Error (19): Illegal character <22> in hex string
Syntax Error (22): Illegal character <6e> in hex string
Syntax Error

O código a seguir foi adaptado de http://brandonrose.org/clustering#Stopwords,-stemming,-and-tokenizing

In [40]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

In [155]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(re.sub('[-\xad][\s\n]','',text)) \
                          for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]{3,}', token):
            filtered_tokens.append(token)
    #stems = [stemmer.stem(t) for t in filtered_tokens]
    #return stems
    return filtered_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rvimieiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rvimieiro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [156]:
tfidf_vectorizer = TfidfVectorizer(input='filename',max_df=0.9, max_features=20000,
                                 min_df=0.05, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))
t = !ls txt2/*.txt
%time tfidf_matrix = tfidf_vectorizer.fit_transform(t)

print(tfidf_matrix.shape)

CPU times: user 1min 24s, sys: 1.53 s, total: 1min 25s
Wall time: 1min 29s
(568, 4407)


In [157]:
terms = tfidf_vectorizer.get_feature_names()
terms[:10]

['aaai',
 'aaai press',
 'aalborg',
 'ability',
 'able',
 'ables',
 'absence',
 'absent',
 'absolute',
 'abstract']

In [32]:
with open('terms.txt','w') as f:
    f.write('\n'.join(terms))

In [158]:
x = tfidf_matrix.toarray()

In [163]:
df = pd.DataFrame(x,columns=terms,index=[re.sub('txt2?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t])
df.head()

,aaai,aaai press,aalborg,ability,able,ables,absence,absent,absolute,abstract,...,y-axis,years,yes,yield,yielding,yields,york,zero,zeros,zhang
1201.2250v1,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.010583,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.024284,0.012901,0.0,0.000000
1201.2706v2,0.000000,0.0,0.0,0.000000,0.009938,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.013044,0.000000,0.0,0.000000
1201.6655v1,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.021812,0.0,...,0.0,0.0,0.0,0.000000,0.010950,0.000000,0.000000,0.000000,0.0,0.000000
1202.1334v2,0.000000,0.0,0.0,0.008395,0.005955,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.016297,0.010620,0.000000,0.000000,0.000000,0.0,0.010219
1202.3698v1,0.025457,0.0,0.0,0.000000,0.000000,0.010693,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.023953,0.007805,0.005548,0.000000,0.000000,0.0,0.000000


In [164]:
doc_term_matrix = pd.merge(df,artigos[['community','id']],left_index=True,right_on='id')
doc_term_matrix.reset_index(inplace=True)
doc_term_matrix.drop('level_0',inplace=True,axis=1)
doc_term_matrix.head()

,aaai,aaai press,aalborg,ability,able,ables,absence,absent,absolute,abstract,...,yes,yield,yielding,yields,york,zero,zeros,zhang,community_y,id
0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.010583,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.024284,0.012901,0.0,0.000000,6,1201.2250v1
1,0.000000,0.0,0.0,0.000000,0.009938,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.013044,0.000000,0.0,0.000000,116,1201.2706v2
2,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.021812,0.0,...,0.0,0.000000,0.010950,0.000000,0.000000,0.000000,0.0,0.000000,151,1201.6655v1
3,0.000000,0.0,0.0,0.008395,0.005955,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.016297,0.010620,0.000000,0.000000,0.000000,0.0,0.010219,151,1202.1334v2
4,0.025457,0.0,0.0,0.000000,0.000000,0.010693,0.000000,0.0,0.000000,0.0,...,0.0,0.023953,0.007805,0.005548,0.000000,0.000000,0.0,0.000000,134,1202.3698v1


In [36]:
doc_term_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Columns: 4265 entries, aaai to id
dtypes: float64(4263), int64(1), object(1)
memory usage: 18.5+ MB


In [37]:
doc_term_matrix.community.unique()

array([  6, 116, 151, 134, 104,  80, 153, 145, 156, 256])

In [166]:
x2 = doc_term_matrix.drop(['id','community_y'],axis=1).\
                            replace(to_replace=0,value=np.NaN).\
                            apply(lambda a: pd.qcut(a,3,labels=['baixo','medio','alto']))
x2 = x2.apply(lambda a: a.cat.add_categories(['zero']))
x2['community_y'] = doc_term_matrix.community_y
x2.head()

,aaai,aaai press,aalborg,ability,able,ables,absence,absent,absolute,abstract,...,years,yes,yield,yielding,yields,york,zero,zeros,zhang,community_y
0,NaN,NaN,NaN,NaN,NaN,NaN,medio,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,alto,medio,NaN,NaN,6
1,NaN,NaN,NaN,NaN,alto,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,alto,NaN,NaN,NaN,116
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alto,NaN,...,NaN,NaN,NaN,medio,NaN,NaN,NaN,NaN,NaN,151
3,NaN,NaN,NaN,medio,medio,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,alto,medio,NaN,NaN,NaN,NaN,medio,151
4,alto,NaN,NaN,NaN,NaN,medio,NaN,NaN,NaN,NaN,...,NaN,NaN,alto,baixo,baixo,NaN,NaN,NaN,NaN,134


In [167]:
x2.fillna('zero').to_csv('doc_term_matrix_tfidf_wo_header_nostem_discrete.csv',header=True,index=False)

In [168]:
freq_vectorizer = CountVectorizer(input='filename',max_df=0.9, max_features=20000,
                                 min_df=0.05, stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))
t = !ls txt2/*.txt
%time freq_matrix = freq_vectorizer.fit_transform(t)

print(freq_matrix.shape)

CPU times: user 1min 35s, sys: 2.5 s, total: 1min 38s
Wall time: 1min 44s
(568, 4407)


In [169]:
terms_freq = freq_vectorizer.get_feature_names()
#with open('terms_tf.txt','w') as f:
#    f.write('\n'.join(terms_freq))

In [170]:
x = freq_matrix.toarray()
df = pd.DataFrame(x,columns=terms_freq,index=[re.sub('txt2?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t])
doc_term_matrix = pd.merge(df,artigos[['community','id']],left_index=True,right_on='id')
doc_term_matrix.set_index('id',inplace=True)
#doc_term_matrix.head()

In [55]:
doc_term_matrix.to_csv('doc_term_matrix_freq.csv',header=True)

In [172]:
filtro = doc_term_matrix.drop('community_y',axis=1)\
                .replace(to_replace=0,value=np.NaN)\
                .apply(lambda a: [-1,0,a.quantile(.75),a.max()])

In [173]:
np.sum(filtro.apply(lambda a: len(np.unique(a))) == 4)

4350

In [175]:
x2 = doc_term_matrix.drop(['community_y'],axis=1)\
            .loc[:,filtro.apply(lambda a: len(np.unique(a))) == 4]\
            .apply(lambda a: pd.cut(a,
                                    [-1,0,a.replace(to_replace=0,value=np.NaN).quantile(.75),a.max()],
                                    labels=['zero','baixo','alto']))
#x2 = x2.apply(lambda a: a.cat.add_categories(['zero']))
x2['community_y'] = doc_term_matrix.community_y
x2.head()

,aaai,aaai press,aalborg,ability,able,ables,absence,absent,absolute,abstract,...,years,yes,yield,yielding,yields,york,zero,zeros,zhang,community_y
id,,,,,,,,,,,,,,,,,,,,,
1201.2250v1,zero,zero,zero,zero,zero,zero,baixo,zero,zero,zero,...,zero,zero,zero,zero,zero,alto,baixo,zero,zero,6
1201.2706v2,zero,zero,zero,zero,baixo,zero,zero,zero,zero,zero,...,zero,zero,zero,zero,zero,baixo,zero,zero,zero,116
1201.6655v1,zero,zero,zero,zero,zero,zero,zero,zero,baixo,zero,...,zero,zero,zero,baixo,zero,zero,zero,zero,zero,151
1202.1334v2,zero,zero,zero,baixo,baixo,zero,zero,zero,zero,zero,...,zero,zero,baixo,baixo,zero,zero,zero,zero,baixo,151
1202.3698v1,alto,zero,zero,zero,zero,baixo,zero,zero,zero,zero,...,zero,zero,alto,baixo,baixo,zero,zero,zero,zero,134


In [176]:
x2.to_csv('doc_term_matrix_freq_wo_header_nostem_discrete.csv',header=True,index=False)

In [98]:
doc_term_matrix.community.value_counts()

104    98
153    95
6      87
134    68
156    56
256    41
80     41
116    35
151    29
145    18
Name: community, dtype: int64

In [113]:
doc_term_matrix.drop('community',axis=1).apply(lambda x: np.corrcoef(x.values,doc_term_matrix.community.values)[0,1])

aaai                             0.014700
aaai press                       0.018226
aalborg                          0.342647
aalborg denmark                  0.319073
abbrevi                          0.063080
abduct                          -0.065507
abil                             0.004500
abl                             -0.005611
abov                            -0.006502
abov equat                      -0.011343
abov exampl                     -0.048061
abov result                     -0.047705
absenc                           0.048683
absent                           0.010926
absolut                          0.014027
absorb                           0.130748
abstract paper                   0.033443
academ                           0.022029
academ press                     0.029569
acceler                         -0.020565
accept                           0.004710
access                          -0.049000
accommod                         0.055704
accomplish                      -0

In [115]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(x2.aaai,x2.community).values)

(61.195164799130595,
 1.3096659985158831e-06,
 18,
 array([[ 64.33098592,  30.31690141,  72.46478873,  25.88028169,
          50.28169014,  13.30985915,  21.44366197,  70.24647887,
          41.4084507 ,  30.31690141],
        [ 17.15492958,   8.08450704,  19.32394366,   6.90140845,
          13.4084507 ,   3.54929577,   5.71830986,  18.73239437,
          11.04225352,   8.08450704],
        [  5.51408451,   2.59859155,   6.21126761,   2.21830986,
           4.30985915,   1.14084507,   1.83802817,   6.02112676,
           3.54929577,   2.59859155]]))

In [120]:
b = x2.drop('community',axis=1).apply(lambda x: chi2_contingency(pd.crosstab(x,x2.community))[1])

KeyboardInterrupt: 

In [132]:
x2[b.sort_values(ascending=True)[200:500].index.values.tolist()+['community']].to_csv('doc_term_matrix_freq_chisq_discrete.csv',index=False)

In [133]:
x3 = pd.read_csv('doc_term_matrix_tfidf_discrete.csv')
x3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Columns: 4264 entries, aaai to community
dtypes: int64(1), object(4263)
memory usage: 18.5+ MB


In [134]:
b3 = x3.drop('community',axis=1).apply(lambda x: chi2_contingency(pd.crosstab(x,x3.community))[1])

In [138]:
x3[b3.sort_values(ascending=True)[300:1000].index.values.tolist()+['community']].to_csv('doc_term_matrix_tfidf_chisq_discrete.csv',index=False)